In [ ]:
## interpolate mag onto t_new
#mag_interp: NDArray[np.float64] = np.column_stack([
#        np.interp(t_new, t_mag, m[:, 0]), #x
#        np.interp(t_new, t_mag, m[:, 1]), #y
#        np.interp(t_new, t_mag, m[:, 2])  #z
#]).astype(np.float64)

## -----------------------------
## Gyro + Accel gravity + Mag correction (no Gravity.csv)
## -----------------------------
#"""
#acc
#"""
#K_acc: float = 0.01
#g0: float = 9.80665
#acc_sigma: float = 1.2 # m/s^2, gate softness around |a|-g0
#g_world_unit: Vec3 = np.array([0.0, 0.0, -1.0], dtype=np.float64)

#q: Quat = q_ref_interp[0].copy()
#q_custom_gyro_acc_mag: NDArray[np.float64] = np.zeros((len(dt), 4), dtype=np.float64)
#g_body_est: NDArray[np.float64] = np.zeros((len(dt), 3), dtype=np.float64)
#a_lin_est: NDArray[np.float64] = np.zeros((len(dt), 3), dtype=np.float64)

#"""
#mag
#"""
## for mag gating (based on norm)
#K_mag: float = 0.011
#m_norm_all: NDArray[np.float64] = np.linalg.norm(mag_interp, axis=1)
#m0: float = np.median(m_norm_all)
#mag_sigma: float = 0.25 * m0
#if mag_sigma < 1e-9:
#        mag_sigma = 1

## build mag_ref_world_h from reference orientation over stable window
#mask = (np.abs(np.linalg.norm(a_net_interp, axis=1) - g0) < 0.3) & (m_norm_all > 1e-6)
#idx = np.where(mask)[0][:3000]   # first 3s

#accum = np.zeros(3)
#cnt = 0
#for i in idx:
#    m_unit = mag_interp[i] / max(np.linalg.norm(mag_interp[i]), 1e-9)
#    m_world = libq.rotate_body_to_world(q_ref_interp[i], m_unit)
#    m_world_h = m_world - np.dot(m_world, g_world_unit) * g_world_unit
#    n = np.linalg.norm(m_world_h)
#    if n > 1e-6:
#        accum += m_world_h / n
#        cnt += 1

#mag_ref_world_h = accum / max(np.linalg.norm(accum), 1e-9)

#"""
#loop
#"""
#for i in range(len(dt)):
#        """
#        gyro
#        """
#        # predict gyro
#        dq: Quat = libq.delta_quat_from_omega(w_avg_gyro[i], dt[i])
#        q_pred: Quat = libq.quat_mul(q, dq)
#        q_pred = libq.quat_norm(q_pred)

#        """
#        acc
#        """
#        # predict grav dir (unit) in body
#        g_pred: Vec3 = libq.rotate_world_to_body(q_pred, g_world_unit)
#        g_pred /= max(float(np.linalg.norm(g_pred)), 1e-9)

#        # accel measurement unit + gate
#        a_unit: Vec3 = a_src_interp[i].copy()
#        a_norm: float = np.linalg.norm(a_src_interp[i])
#        if a_norm < 1e-9:
#                a_unit = g_pred.copy()
#        else:
#                a_unit = a_unit / a_norm

#        # accel trust gating: if |a| deviates from g0, trust less
#        dev_acc: float = abs(a_norm - g0)
#        # w in [0,1]. 1 near static, 0 high linear acceleration
#        acc_weight: float = np.exp(-0.5 * (dev_acc / acc_sigma) ** 2)        

#        # gravity correction
#        err_axis_acc: Vec3 = np.cross(g_pred, a_unit)

#        """
#        mag
#        """
#        # mag measurement unit + gate
#        m_unit: Vec3 = mag_interp[i].copy()
#        m_norm: float = np.linalg.norm(m_unit)

#        mag_weight: float = None
#        err_axis_mag: Vec3 = None
#        if m_norm < 1e-9:
#                mag_weight = 0
#                err_axis_mag = np.array([0.0, 0.0, 0.0], dtype=np.float64)
#        else:
#                m_unit = m_unit / m_norm

#                # gate by magnitude deviation
#                dev_mag: float = abs(m_norm - m0)
#                # w in [0,1]. 1 near static, 0 high linear acceleration
#                mag_weight: float = np.exp(-0.5 * (dev_mag / mag_sigma) ** 2)
                
#                # tilt compensation: keep only horizontal component (remove along gravity)
#                mag_body_h: Vec3 = m_unit - np.dot(m_unit, g_pred) * g_pred
#                mag_body_h /= max(np.linalg.norm(mag_body_h), 1e-9)
#                if np.linalg.norm(mag_body_h) < 0.2:
#                        mag_weight = 0

#                # predicted horizontal mag in body = R^T * mag_ref_world_h
#                mag_pred: Vec3 = libq.rotate_world_to_body(q_pred, mag_ref_world_h)
#                mag_pred_h: Vec3 = mag_pred - np.dot(mag_pred, g_pred) * g_pred
#                mag_pred_h /= max(np.linalg.norm(mag_pred_h), 1e-9)
#                #err_axis_mag = np.cross(mag_pred_h, mag_body_h)
#                err_axis_mag = np.cross(mag_body_h, mag_pred_h)

#        """
#        combine corrections
#        """
#        err_axis: Vec3 = K_acc * acc_weight * err_axis_acc + K_mag * mag_weight * err_axis_mag
#        dq_corr: Quat = np.array([
#                1.0,
#                0.5 * err_axis[0],
#                0.5 * err_axis[1],
#                0.5 * err_axis[2]], dtype=np.float64)
#        dq_corr = libq.quat_norm(dq_corr)

#        q = libq.quat_mul(q_pred, dq_corr)
#        q = libq.quat_norm(q)
#        q_custom_gyro_acc_mag[i] = q

#        # estimate gravity in body frame / linear accel
#        g_body_est[i] = libq.rotate_world_to_body(q, g_world_unit) * g0
#        a_lin_est[i] = a_net_interp[i] + g_body_est[i]

#angle_err_gyro_acc_mag: NDArray[np.float64] = libq.calc_angle_err(q_custom_gyro_acc_mag, q_ref_interp)

#print("Gyro+Accel+Mag angle_err (rad): min/max/mean",
#      float(angle_err_gyro_acc_mag.min()),
#      float(angle_err_gyro_acc_mag.max()),
#      float(angle_err_gyro_acc_mag.mean()))
#print("Gyro+Accel+Mag angle_err (deg): min/max/mean",
#      float(np.rad2deg(angle_err_gyro_acc_mag.min())),
#      float(np.rad2deg(angle_err_gyro_acc_mag.max())),
#      float(np.rad2deg(angle_err_gyro_acc_mag.mean())))